In [1]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor

In [2]:

Nimply = namedtuple("Nimply", "row, num_objects")

In [3]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [4]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [5]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [6]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result


def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

In [7]:
def optimal_startegy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

In [94]:
def make_strategy1(genome: dict) -> Callable:
    def evolvable1(state: Nim) -> Nimply:
        data = cook_status(state)

        if random.random() < genome["p"]:
            ply = Nimply(data["shortest_row"], random.randint(1, state.rows[data["shortest_row"]]))
        else:
            ply = Nimply(data["longest_row"], random.randint(1, state.rows[data["longest_row"]]))

        return ply

    return evolvable1

In [85]:
def make_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)
        #print(f'righe attive:{data["active_rows_number"]}')
        if data["active_rows_number"] > 3:
            if random.random() < genome["p"]:
                ply = Nimply(data["shortest_row"], random.randint(1, state.rows[data["shortest_row"]]))
            else:
                ply = Nimply(data["longest_row"], random.randint(1, state.rows[data["longest_row"]]))

            return ply
        else:
            #print(f'presi: {state.rows[data["longest_row"]]-1}')
            if state.rows[data["longest_row"]] > 1:
                ply = Nimply(data["longest_row"],state.rows[data["longest_row"]]-1)
            else:
                ply = Nimply(data["longest_row"],1)
            return ply

    return evolvable

In [87]:
logging.getLogger().setLevel(logging.DEBUG)

strategy = (make_strategy({"p": 0.5}), pure_random)

nim = Nim(5)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")


DEBUG:root:status: Initial board  -> <1 3 5 7 9>
DEBUG:root:status: After player 0 -> <1 3 5 7 1>
DEBUG:root:status: After player 1 -> <1 0 5 7 1>
DEBUG:root:status: After player 0 -> <0 0 5 7 1>
DEBUG:root:status: After player 1 -> <0 0 3 7 1>
DEBUG:root:status: After player 0 -> <0 0 3 1 1>
DEBUG:root:status: After player 1 -> <0 0 1 1 1>
DEBUG:root:status: After player 0 -> <0 0 0 1 1>
DEBUG:root:status: After player 1 -> <0 0 0 1 0>
DEBUG:root:status: After player 0 -> <0 0 0 0 0>
INFO:root:status: Player 0 won!


In [99]:
NUM_MATCHES = 10
NIM_SIZE = 10


def evaluate(strategy: Callable) -> float:
    opponent = (strategy,make_strategy1({"p":0.5}))
    won = 0

    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        while nim:
            ply = opponent[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            won += 1
    return won / NUM_MATCHES

In [100]:
evaluate(make_strategy({"p": 0.5}))

0.5